In [14]:
import pandas as pd
import geopandas as gpd

In [15]:
from pathlib import Path
import re

files = []

for path in Path('imagens').rglob('*.*'):
    for i in re.findall(r"([A-Z]{2,4}-[0-9]{2})", path.name):
        files.append([i, path.as_posix()])
#     print(re.findall(r"([A-Z]{2,4}-[0-9]{2})", path.name), path )

In [16]:
df_imagens = pd.DataFrame(files, columns=['cod', 'path'])

In [17]:
df_cods = pd.DataFrame(df_imagens.groupby('cod')['path'].apply(list))

In [18]:
df_imagens = pd.DataFrame(list(map(lambda x: x, df_cods.path))).add_prefix('imagem_')

In [19]:
df_imagens['cod'] = df_cods.index

In [20]:
df_imagens.set_index('cod')

,imagem_0,imagem_1,imagem_2,imagem_3,imagem_4,imagem_5,imagem_6,imagem_7
cod,,,,,,,,
AUR-00,imagens/BOTUJURU/EQUIPAMENTOS NOVOS/AUR-00 Pra...,imagens/BOTUJURU/EQUIPAMENTOS NOVOS/AUR-00 Pra...,None,None,None,None,None,None
AUR-01,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,None,None,None,None
AUR-02,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,None,None,None,None
AUR-03,imagens/MONTANHÃO/AUR-03 Praça Henrique Trinda...,imagens/MONTANHÃO/AUR-03 Praça Henrique Trinda...,imagens/MONTANHÃO/AUR-03 Praça Henrique Trinda...,None,None,None,None,None
AUR-04,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,None,None
...,...,...,...,...,...,...,...,...
SAU-15,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,None,None
SAU-16,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,None,None
SAU-17,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,None,None


In [21]:
df_equip = pd.read_csv('./GIS/planilhas/equipamentos-XY.csv', delimiter=';', decimal=',')

In [22]:
df_equip['cod'] = df_equip['COD DO LOCAL ']
# df_equip.columns

In [23]:
df_equip = pd.merge(df_equip, df_imagens.set_index('cod'), on='cod', how='left')

In [24]:
gdf_equip = gpd.GeoDataFrame(df_equip, geometry=gpd.points_from_xy(df_equip.X, df_equip.Y))

In [26]:
gdf_equip.drop(['X', 'Y', 'COD DO LOCAL ',  ], axis=1).to_file('GIS/equipamentos-com-fotos.gpkg', driver='GPKG')

In [27]:
for b in gdf_equip.BAIRRO.unique():
    b_formatado = b.lower().replace(" ", "_")
    gdf_equip.drop(['X', 'Y', 'COD DO LOCAL ',  ], axis=1).loc[gdf_equip.BAIRRO == b].to_file(f'GIS/equipamentos-com-fotos-{b_formatado}.gpkg', driver='GPKG')